### 1. INITIALIZE THINGS

In [1]:
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interactive_output, Layout, interact_manual, interact
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cartopy import crs, feature

from utils import load_teseo_particles_txt

# items_layout = Layout( width='auto')

directory_box = widgets.Text(
    value="/home/aragong/repositories/plotting-notebooks/mock/TESEO_TOOLS_459612f0",
    placeholder='Type something',
    description='Directory:',
    disabled=False
)

pattern_box = widgets.Select(
    options=['*_particles_*.txt', '*_properties_*.txt', '*_grid_*.txt'],
    description='File-pattern:',
    disabled=False
)
# widgets.Text(
#     placeholder='Type something',
#     description='File-pattern:',
#     disabled=False
# )

dx_slider = widgets.FloatSlider(
    value=1.000,
    min=0.000,
    max=2.000,
    step=0.02,
    description='Zoom out:',
    readout_format='.3f',
)

psize_slider =  widgets.IntSlider(
    value=1,
    min=1,
    max=50,
    step=1,
    description='Particle size:',
)

grid_checkbox = widgets.Checkbox(
    value=True,
    description='Gridlines',
    disabled=False,
    indent=False
)

pmarker_box = widgets.Dropdown(
    options=[("point", "o"), ("cross", "x"), ("square", "s")],
    value="o",
    description='Marker',
)

pcolor_box = widgets.Dropdown(
    options=["blue", "red", "black", "green", "yellow", "brown"],
    value="blue",
    description='Color',
)

### 2. ADD DIRECTORY AND FILES-PATTERN

```python
directory = '/home/aragong/repositories/plotting-notebooks/mock/TESEO_TOOLS_459612f0'
pattern = '*particles*.txt'
grid = '/home/aragong/repositories/plotting-notebooks/mock/TESEO_TOOLS_459612f0/input/grid.dat'
```

In [2]:
@interact(directory_str=directory_box, pattern_str=pattern_box)
def set_paths(directory_str, pattern_str):
    directory_box.value = directory_str
    pattern_box.value = pattern_str

interactive(children=(Text(value='/home/aragong/repositories/plotting-notebooks/mock/TESEO_TOOLS_459612f0', de…

### 3. LOAD FILES INTO DATAFRAME

In [3]:
directory = directory_box.value
pattern = pattern_box.value

print(f"Loading files in folder: '{directory}' with pattern: '{pattern}'")

if pattern_box.value == "*_particles_*.txt":
    df = load_teseo_particles_txt(pattern, directory)
if pattern_box.value == "*_properties_*.txt":
    print("TODO - read properties")
if pattern_box.value == "*_grid_*.txt":
    print("TODO - read grid")

Loading files in folder: '/home/aragong/repositories/plotting-notebooks/mock/TESEO_TOOLS_459612f0' with pattern: '*_particles_*.txt'


In [15]:

play = widgets.Play(
    value=df.time.unique().min(),
    min=df.time.unique().min(),
    max=df.time.unique().max(),
    step=np.unique(np.diff(df.time.unique())),
    interval=500,
    description="Press play",
    disabled=False,
)

time_slider = widgets.FloatSlider(
    value=df.time.unique().min(),
    min=df.time.unique().min(),
    max=df.time.unique().max(),
    step=np.unique(np.diff(df.time.unique())),
    description='Time (h):',
    readout_format='.1f',
)
widgets.jslink((play, 'value'), (time_slider, 'value'))


def plot_teseo_particles(t, dx, pmarker, pcolor, psize, grid,  gshhs=None, extent=None):
    
    fig = plt.figure(figsize=[15, 10])
    ax = fig.add_subplot(1, 1, 1, projection=crs.PlateCarree())
    
    df_tmp = df.loc[df.time == t]

    if dx is not None:
        extent = [df_tmp.lon.min()-dx, df_tmp.lon.max()+dx, df_tmp.lat.min()-dx, df_tmp.lat.max()+dx]



    if extent is not None:
        ax.set_extent(extent, crs=crs.PlateCarree())
    if gshhs:
        ax.add_feature(feature.GSHHSFeature(scale="full")) 
    
    gl = ax.gridlines(crs=crs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='black', alpha=0.8, linestyle='--')
    
    gl.top_labels = False
    gl.left_labels = False
    
    if grid is False:
        gl.xlines = False
        gl.ylines = False

    ax.add_feature(feature.LAND)
    ax.add_feature(feature.OCEAN)
    ax.add_feature(feature.COASTLINE)
    ax.add_feature(feature.BORDERS, linestyle=":")
    ax.add_feature(feature.LAKES, alpha=0.5)
    ax.add_feature(feature.RIVERS)
    ax.scatter(df_tmp.lon, df_tmp.lat, marker=pmarker, c=pcolor, s=psize)

    ax.set_adjustable('datalim')
    

### 4. ORGANIZE PANELS AND RUN VISUALIZATION

In [16]:
time_panel = widgets.HBox([play, time_slider])
# control_panel = widgets.HBox([widgets.VBox([grid_checkbox, dx_slider, psize_slider]), widgets.VBox([pmarker_box, pcolor_box])])
tab = widgets.Tab()
tab.children = [widgets.VBox([grid_checkbox, dx_slider]), widgets.VBox([psize_slider, pmarker_box, pcolor_box])]
tab._titles = {0:"map properties", 1:"particle properties"}

out = interactive_output(plot_teseo_particles, {"t":time_slider, "dx":dx_slider, "pmarker":pmarker_box, "pcolor": pcolor_box, "psize": psize_slider, "grid":grid_checkbox})
display(tab, out, time_panel)

Output()